# add features
# window data

In [1]:
import numpy as np
import argparse

import os
import math
import json

import pandas as pd
from sklearn.linear_model import LinearRegression

from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures


import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
matplotlib.rc( 'savefig', facecolor = 'white' )
from matplotlib import pyplot
%matplotlib inline

i_seed = 0

import sys
sys.path.append('../') # add this line so Data and data are visible in this file
sys.path.append('../../') # add this line so Data and data are visible in this file
sys.path.append('../PatchWand/') # add this line so Data and data are visible in this file

# from PatchWand import *
from filters import *
from setting import *
from segmentation import *
# from stage1_patch import *
# from TimeStampReader import *
# from Subject import *
# from PatchParser import *
from preprocessing import *
from ECG_module import *
# from PPG_module import *
from evaluate import *
from stage1_cosmed import *
from stage3_preprocess import *
from dataIO import *

from importlib import reload
%load_ext autoreload
%autoreload 2

In [2]:
parser = argparse.ArgumentParser(description='SpO2_estimate')
parser.add_argument('--input_folder', metavar='input_folder', help='input_folder',
                    default='../')
parser.add_argument('--output_folder', metavar='output_folder', help='output_folder',
                    default='../')
# parser.add_argument('--subject_id', metavar='subject_id', help='subject_id',
#                     default='101')
parser.add_argument('--training_params_file', metavar='training_params_file', help='training_params_file',
                    default='training_params_list.json')


# checklist 3: comment first line, uncomment second line
args = parser.parse_args(['--input_folder', '../../data/stage2/', 
                          '--output_folder', '../../data/stage3/',
#                           '--subject_id', 'sub118',
                          '--training_params_file', 'training_params.json',
                         ])
# args = parser.parse_args()
print(args)

Namespace(input_folder='../../data/stage2/', output_folder='../../data/stage3/', training_params_file='training_params.json')


In [3]:
Fs = FS_RESAMPLE
Fs

250

In [4]:
inputdir = args.input_folder
outputdir = args.output_folder
# subject_id = args.subject_id

if not os.path.exists(outputdir):
    os.makedirs(outputdir)
    
# outputdir_sub = outputdir+subject_id+'/'
# if not os.path.exists(outputdir_sub):
#     os.makedirs(outputdir_sub)
    
training_params_file = args.training_params_file


In [5]:
with open(training_params_file) as json_file:
    training_params_list = json.load(json_file)
    
training_params = training_params_list[0]

In [6]:
training_params

{'list_signal': ['ECG', 'accelZ', 'ppg_r_1'],
 'list_feature': ['VE_cosmed', 'HR_patch'],
 'list_output': ['RR_cosmed',
  'VT_cosmed',
  'EE_cosmed',
  'SPO2_cosmed',
  'HR_cosmed',
  'VO2_cosmed',
  'resp_cosmed'],
 'list_meta': ['subject_id', 'task'],
 'FS_RESAMPLE_DL': 100,
 'window_size': 60,
 'overlap': 0.9}

In [7]:
list_signal = training_params['list_signal']
list_feature = training_params['list_feature']
list_output = training_params['list_output']
list_meta = training_params['list_meta']
FS_RESAMPLE_DL = training_params['FS_RESAMPLE_DL']
window_size = training_params['window_size'] # s
overlap = training_params['overlap'] # out of 1

In [8]:
# if cosmed_unfiltered:
#     obj_cosmed = load_sub('../../data/stage1/'+subject_id+'/cosmed_unfiltered/data')
# else:

# obj_cosmed = load_sub('../../data/stage1/'+subject_id+'/cosmed/data')



In [9]:
stationary_names = ['Baseline 0']
for j in range(5):
    stationary_names.append('Recovery ' + str(j))
# stationary_names.append( '6MWT 0' )
# stationary_names.append( '6MWT-R 0' )
# stationary_names.append( 'Stair 0' )
# stationary_names.append( 'Walk 0' )
# stationary_names.append( 'Run 0' )

stationary_names

['Baseline 0',
 'Recovery 0',
 'Recovery 1',
 'Recovery 2',
 'Recovery 3',
 'Recovery 4']

# get sync data

## change activity name
## maybe remove scaling

In [10]:
for i in range(21):
    print(i+101)
#     subject_id = 'sub'+str(i+101)
    

101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121


105 ECG
108 ECG
109 ECG
112 cosmed
115

In [11]:
no_scaling = False

df_sync = []
# for i in range(21):

for i in range(300):
    subject_id = 'sub'+str(i)

#     if subject_id =='sub105' or subject_id =='sub108' or subject_id =='sub109' or subject_id =='sub112' or subject_id =='sub117':
    if subject_id =='sub112':
        continue
        

#     if subject_id !='sub106':
#         print(subject_id)
#         continue
    
    inputdir_sub = inputdir+subject_id+'/'
    if subject_id not in os.listdir(inputdir):
        continue
        
        
    print('subject_id is ', subject_id)

    for syn_file in os.listdir(inputdir_sub):
        if '.feather'  in syn_file:
            df_sub = pd.read_feather(inputdir_sub+syn_file)
#             df_sub = filter_DFcolumns(df_sub.copy(), Fs)

#     sys.exit()
    # get raw task names except for Transition
    task_unique = df_sub['task'].unique()
    task_unique = task_unique[task_unique!='Transition']

#     df_sub = filter_DFcolumns(df_sub, Fs)
        
    for stationary_name in stationary_names:
        
        if 'Baseline' not in stationary_name and 'Recovery' not in stationary_name:
            stationary_name_new = stationary_name.split(' ')[0]
        else:

            # get recovery index
            i_recovery = np.where(task_unique==stationary_name)[0]

            # the index of the task associated with this recovery is i_recovery-1. Get its name
            task_name = task_unique[i_recovery-1][0]

            # spell out new recovery name
    #         recovery_name_new = ' '.join([recovery_name.split(' ')[0], task_name.split(' ')[0] ,recovery_name.split(' ')[1]])

            if 'Baseline' in stationary_name:
                stationary_name_new = stationary_name.split(' ')[0]
            else:
                stationary_name_new = ' '.join([stationary_name.split(' ')[0], task_name.split(' ')[0]])
        

        df_sub.loc[df_sub['task']==stationary_name, 'task'] = stationary_name_new
        
        if no_scaling:
            obj_cosmed = load_sub('../../data/stage1/'+subject_id+'/cosmed/data')
            print(obj_cosmed.Weight)
            df_sub['VT_cosmed'] = df_sub['VT_cosmed'] * obj_cosmed.Weight
            df_sub['VE_cosmed'] = df_sub['VE_cosmed'] * obj_cosmed.Weight
            df_sub['VO2_cosmed'] = df_sub['VO2_cosmed'] * obj_cosmed.Weight
            df_sub['VCO2_cosmed'] = df_sub['VCO2_cosmed'] * obj_cosmed.Weight
            df_sub['EE_cosmed'] = df_sub['EE_cosmed'] * obj_cosmed.Weight

    df_sub = df_sub.loc[(df_sub['task'].str.contains("Baseline", case=False)) | (df_sub['task'].str.contains("Recovery", case=False)) | 
                        (df_sub['task']=="6MWT") | (df_sub['task']=="6MWT-R") | (df_sub['task']=="Run") | (df_sub['task']=="Walk") | (df_sub['task']=="Stair")
                       ].reset_index(drop=True)
#     df_sub = df_sub.loc[(df_sub['task'].str.contains("Baseline", case=False))].reset_index(drop=True)
    df_sub['subject_id'] = subject_id[3:]
    
    print(df_sub['task'].unique())

    df_sync.append(df_sub.copy())
    #     print((df_sync['task']==recovery_name).sum())
    
df_sync = pd.concat(df_sync).reset_index(drop=True)

subject_id is  sub101
['Baseline' 'Recovery 6MWT' 'Recovery 6MWT-R' 'Recovery Stair'
 'Recovery Walk' 'Recovery Run']
subject_id is  sub102
['Baseline' 'Recovery 6MWT' 'Recovery 6MWT-R' 'Recovery Stair'
 'Recovery Walk' 'Recovery Run']
subject_id is  sub103
['Baseline' 'Recovery 6MWT' 'Recovery 6MWT-R' 'Recovery Stair'
 'Recovery Walk' 'Recovery Run']
subject_id is  sub104
['Baseline' 'Recovery 6MWT-R' 'Recovery 6MWT' 'Recovery Stair'
 'Recovery Walk' 'Recovery Run']
subject_id is  sub105
['Baseline' 'Recovery 6MWT' 'Recovery 6MWT-R' 'Recovery Stair'
 'Recovery Walk' 'Recovery Run' 'Recovery 5']
subject_id is  sub106
['Baseline' 'Recovery 6MWT-R' 'Recovery 6MWT' 'Recovery Stair'
 'Recovery Walk' 'Recovery Run' 'Recovery 5']
subject_id is  sub107
['Baseline' 'Recovery 6MWT-R' 'Recovery 6MWT' 'Recovery Stair'
 'Recovery Walk' 'Recovery Run']
subject_id is  sub108
['Baseline' 'Recovery 6MWT' 'Recovery 6MWT-R' 'Recovery Stair'
 'Recovery Walk' 'Recovery Run']
subject_id is  sub109
['Baseli

# add HR_patch and resp_cosmed

In [12]:
df_sync['HR_patch'] = 0
df_sync['resp_cosmed'] = 0

include_spectral = False

for subject_id in df_sync['subject_id'].unique():


    df_sub = df_sync[df_sync['subject_id']==subject_id]
    outputdir_sub = outputdir+subject_id+'/'
    
    for task_name in df_sub['task'].unique():
        
#         if subject_id!='111':
#             continue
#         if task_name!='Recovery Run':
#             continue
        print('sub: {}, task: {}'.format(subject_id, task_name))
        df_task = df_sub[df_sub['task']==task_name]
        ECG_raw_patch = df_task['ECG'].values
        
        fig_name = 'ECG_diagnostics_'+task_name
        QRS_detector_dict_patch = task_HR_detector(ECG_raw_patch, Fs, fig_name=fig_name, outputdir=outputdir_sub, show_plot=False)
        
#         sys.exit()
        fig_name = 'physio_diagnostics_'+task_name
        plot_inputsigs(df_task, Fs, fig_name=fig_name, outputdir=outputdir_sub, show_plot=False)
#         if subject_id=='102':
#         sys.exit()

        def get_RRVT2resp(df_task, Fs):
            f_arr_raw = df_task['RR_cosmed'].values/60
            t_arr = np.arange(df_task.shape[0])/Fs
            # smooth it since it's usally digital and disgusting
            f_arr = get_smooth(f_arr_raw, N=Fs*5)
            # v_arr is the simulated sinusoidal breath signal, sampled eqaully as t_arr
            v_arr, f_sim_interp = get_sim_breath(t_arr, f_arr, downsample_factor=Fs//5) # Fs=500Hz
            resp_cosmed = v_arr*df_task['VT_cosmed']/2
            
            return resp_cosmed

        resp_cosmed = get_RRVT2resp(df_task, Fs)
        
        t_ecg = np.arange(ECG_raw_patch.shape[0])/Fs
        hr_interp = np.interp(t_ecg, QRS_detector_dict_patch['ts_hr'], QRS_detector_dict_patch['hr'])
        O2pulse_cosmedpatch = df_task['VO2_cosmed']/hr_interp

        
#         sys.exit()
        df_sync.loc[(df_sync['subject_id']==subject_id) & (df_sync['task']==task_name), 'HR_patch'] = hr_interp
        df_sync.loc[(df_sync['subject_id']==subject_id) & (df_sync['task']==task_name), 'resp_cosmed'] = resp_cosmed
        df_sync.loc[(df_sync['subject_id']==subject_id) & (df_sync['task']==task_name), 'O2pulse_cosmedpatch'] = O2pulse_cosmedpatch

        
        if include_spectral:
            i_peaks = QRS_detector_dict_patch['i_beat_peaks']
            features_spectral = get_spectral_features(df_task, i_peaks, Fs)

            for key, value in features_spectral.items():
                df_sync.loc[(df_sync['subject_id']==subject_id) & (df_sync['task']==task_name), key] = value


sub: 101, task: Baseline
	 use S peaks
	 use S peaks
sub: 101, task: Recovery 6MWT
	 use S peaks
	 use S peaks
sub: 101, task: Recovery 6MWT-R
	 use S peaks
	 use S peaks
sub: 101, task: Recovery Stair
	 use S peaks
	 use S peaks
sub: 101, task: Recovery Walk
	 use S peaks
	 use S peaks
sub: 101, task: Recovery Run
	 use S peaks
	 use S peaks
sub: 102, task: Baseline
	 use S peaks
	 use S peaks
sub: 102, task: Recovery 6MWT
	 use S peaks
	 use S peaks
sub: 102, task: Recovery 6MWT-R
	 use S peaks
	 use S peaks
sub: 102, task: Recovery Stair
	 use S peaks
	 use S peaks
sub: 102, task: Recovery Walk
	 use S peaks
	 use S peaks
sub: 102, task: Recovery Run
	 use S peaks
	 use S peaks
sub: 103, task: Baseline
	 use S peaks
	 use S peaks
sub: 103, task: Recovery 6MWT
	 use S peaks
	 use S peaks
sub: 103, task: Recovery 6MWT-R
	 use S peaks
	 use S peaks
sub: 103, task: Recovery Stair
	 use S peaks
	 use S peaks
sub: 103, task: Recovery Walk
	 use S peaks
	 use S peaks
sub: 103, task: Recove

In [ ]:
# df = df_sync.loc[(df_sync['subject_id']==subject_id) & (df_sync['task']==task_name)]

In [ ]:
df_sub = df_sync[df_sync['subject_id']=='101'].copy()
df_sub

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1,figsize=(5,5), dpi=100)
ts= np.arange(df_sub.shape[0])/Fs
ax1.plot(ts, df_sub['O2pulse_cosmedpatch'])
ax1.set_ylabel('O2pulse_cosmedpatch\n[ml/beat]')
ax2.plot(ts, df_sub['task'])
ax2.set_xlabel('time (s)')

In [ ]:
# df_sub = df_sync[df_sync['subject_id']==101]

# for task_name in df_sub['task'].unique():

#     df_task = df_sub[df_sub['task']==task_name]

In [ ]:
# df_task

In [ ]:
# aaa = filter_DFcolumns(df_task.copy(), Fs)


In [ ]:
# aaa['ppg_g_1'].plot()

In [ ]:
df_filterd = []

for subject_id in df_sync['subject_id'].unique():


    df_sub = df_sync[df_sync['subject_id']==subject_id]
    
    for task_name in df_sub['task'].unique():
        
        df_task = df_sub[df_sub['task']==task_name]
        df_task = filter_DFcolumns(df_task.copy(), Fs)
        
        df_filterd.append(df_task)
        
df_filterd = pd.concat(df_filterd).reset_index(drop=True)

In [ ]:
# aaa = filter_DFcolumns(df_task.copy(), Fs)

# plt.plot(aaa['ppg_r_1'])

In [ ]:
show_correlation = False

if show_correlation:

    import seaborn as sns

    for subject_id in df_sync['subject_id'].unique():
        fig, ax = plt.subplots(1,1, figsize=(8,8), dpi=80)

        df_sub = df_sync[df_sync['subject_id']==subject_id]
        df_sub = df_sub[df_sub['Sampled_cosmed']==1]

        sns.scatterplot(data=df_sub, x="VE_cosmed", y="EE_cosmed", hue="task", ax=ax)


# check N_samples, focus on important signals, features, output, and meta

In [ ]:
N_samples = df_filterd.shape[0]
print('there are {} sampels ({:.2f} min) in all subjects'.format(N_samples, N_samples/Fs/60))

In [ ]:
print('list_signal', list_signal)
print('list_feature', list_feature)
print('list_output', list_output)
print('list_meta', list_meta)

In [ ]:
# df = df_sync[list_signal+list_feature+list_output+list_meta]
df = df_filterd[list_signal+list_feature+list_output+list_meta]



# windowing the data 

In [ ]:
# df_sync['HR_patch'] = 0

# for subject_id in df_sync['subject_id'].unique():


#     df_sub = df_sync[df_sync['subject_id']==subject_id]
#     outputdir_sub = outputdir+subject_id+'/'
    
#     for task_name in df_sub['task'].unique():
        
# #         if subject_id!='111':
# #             continue
# #         if task_name!='Recovery Run':
# #             continue
#         print('sub: {}, task: {}'.format(subject_id, task_name))
#         df_task = df_sub[df_sub['task']==task_name]
#         ECG_raw_patch = df_task['ECG'].values
        
#         fig_name = 'ECG_diagnostics_'+task_name
#         QRS_detector_dict_patch = task_HR_detector(ECG_raw_patch, Fs, fig_name=fig_name, outputdir=outputdir_sub, show_plot=False)
        
#         fig_name = 'physio_diagnostics_'+task_name
#         plot_inputsigs(df_task, fig_name=fig_name, outputdir=outputdir_sub, show_plot=False)
# #         if subject_id=='102':
# #         sys.exit()
        
#         t_ecg = np.arange(ECG_raw_patch.shape[0])/Fs
#         hr_interp = np.interp(t_ecg, QRS_detector_dict_patch['ts_hr'], QRS_detector_dict_patch['hr'])
        
#         df_sync.loc[(df_sync['subject_id']==subject_id) & (df_sync['task']==task_name), 'HR_patch'] = hr_interp

In [ ]:
# pick only the stationary segment
df_Baseline = df.loc[df['task'].str.contains("Baseline", case=False)]
df_Recovery = df.loc[df['task'].str.contains("Recovery", case=False)]

df_Movement = df[df['task']=='6MWT']
df_Stationary = pd.concat([df_Baseline,df_Recovery, df_Movement])

N_samples = df_Stationary.shape[0]
print('there are {} sampels ({:.2f} min)'.format(N_samples, N_samples/Fs/60))

data_all = []
feature_all = []
label_all = []
meta_all = []



for subject_id in df_Stationary['subject_id'].unique():
    
    print('working on', subject_id)
    
    obj_cosmed = load_sub('../../data/stage1/sub'+str(subject_id)+'/cosmed/data')
    weight = obj_cosmed.Weight

    df_sub = df_Stationary[df_Stationary['subject_id']==subject_id].copy()

    
    task_name = 'Baseline'
    df_task = df_sub[df_sub['task']==task_name].reset_index(drop=True).copy()
    
    # normalize HR based on Altini's method
    
    HR_baseline = df_task['HR_patch'].mean()
    HR_range = df_sub['HR_patch'].max() - df_sub['HR_patch'].min()
    
    df_sub['HR_patch'] = (df_sub['HR_patch'] - HR_baseline ) / HR_range
    
    
    
    for task_name in df_sub['task'].unique():
        
        
        if (subject_id=='110') & (task_name=='Recovery 6MWT-R'):
            continue
        
    #     print(task_name)
        if task_name == 'Recovery 5':
            continue
        df_task = df_sub[df_sub['task']==task_name].reset_index(drop=True).copy()
    #     df_task
        N = df_task.shape[0]

        window_length = int(window_size*Fs)  # windown length in number of samples
    #     window_length, overlap
        N_overlap = int(window_length*overlap)
        # int(60*0.8*Fs) = 4.8 sec * Fs

        N_windows = (N-N_overlap) // (window_length-N_overlap)
        i_starts = np.arange(0,N_windows)*(window_length-N_overlap)
        i_ends = i_starts + window_length
        print('\t{} has {} windows'.format(task_name, N_windows) )

#         print(df_task['subject_id'].unique())
    #     if 'Run' in task_name:
    #         sys.exit()
#         sys.exit()
    #     print()
        for (i_start, i_end) in zip(i_starts, i_ends):
            df_window = df_task[i_start:i_end].copy()

            data = get_data_condensed(df_window, list_signal, Fs, FS_RESAMPLE_DL)
    #         data = df_window[list_input].values
            feature = get_feature_condensed(df_window, list_feature)
            feature = np.r_[feature, weight]
    #         data = get_data_condensed(data)

            label = get_label_condensed(df_window, list_output)
            meta = get_meta_condensed(df_window, list_meta)

    #         sys.exit()

    #         label = df_window[list_output].values

    #         print(data.shape)
            data_all.append(data)
            feature_all.append(feature)
            label_all.append(label)
            meta_all.append(meta)


feature_all = np.stack(feature_all)
label_all = np.stack(label_all)

data_all = np.stack(data_all)
data_all = data_all.transpose(0, 2, 1)


meta_all = np.stack(meta_all)

tasks_dict = {
    'Baseline': 0, 
    'Recovery 6MWT': 1, 
    'Recovery 6MWT-R': 2, 
    'Recovery Stair': 3, 
    'Recovery Walk': 4,
    'Recovery Run': 5,
    '6MWT': 6,
}

np.unique(meta_all[:,1])
task_temp = np.zeros(meta_all.shape[0])
for j, meta_task in enumerate(meta_all[:,1]):
    task_temp[j] = tasks_dict[meta_task]

meta_all = np.c_[meta_all[:,0].astype(int), task_temp].astype(int)

In [ ]:
list_feature.append('weight')

In [ ]:
list_feature

# LOSO CV

In [ ]:
# # data_all[:,1,:].mean(axis=1, keepdims=True).shape
# data_all[:,[1],:].shape

# data_all[:,[1],:].mean(axis=-1).shape

In [ ]:
# data_all[:,i_data,:].mean(axis=-1)[mask_train].shape

feature_all.shape

In [ ]:
# model = LinearRegression()

est_concat = []
label_concat = []

# i_label = list_output.index('VO2_cosmed')
# i_data = list_input.index('HR_patch')

label_name = ['EE_cosmed']
# data_name = ['HR_patch', 'VE_cosmed']
# feature_name = ['HR_patch', 'VE_cosmed']
feature_name = ['HR_patch', 'VE_cosmed', 'weight']
# feature_name = ['HR_patch', 'weight']
# feature_name = ['weight']


# i_label = list_output.index(label_name)
# i_data = list_input.index(data_name)
i_label = [i for i,l in enumerate(list_output) if l in label_name]
i_feature = [i for i,l in enumerate(list_feature) if l in feature_name]


df_performance = []

for subject_id in np.unique( meta_all[:,0]):
    print('working on sub', subject_id)
    
    model = LinearRegression()

    mask_train = meta_all[:,0]!=subject_id
    mask_val = meta_all[:,0]==subject_id
    
#     mask_train = (meta_all[:,0]==subject_id) & (meta_all[:,1]<3)
#     mask_val = (meta_all[:,0]==subject_id) & (meta_all[:,1]>=3) & (meta_all[:,1]!=5)

#     mask_train = (meta_all[:,0]!=subject_id) & (meta_all[:,1]!=5)
#     mask_val = (meta_all[:,0]==subject_id) & (meta_all[:,1]!=5)



#     data_train = data_all[:,i_data,:].mean(axis=1)[mask_train][:,None]
    feature_train = feature_all[:,i_feature][mask_train]
    label_train = label_all[mask_train, i_label]
    meta_train = meta_all[mask_train, :]

#     data_val = data_all[:,i_data,:].mean(axis=1)[mask_val][:,None]
    feature_val = feature_all[:,i_feature][mask_val]
    label_val = label_all[mask_val, i_label]
    meta_val = meta_all[mask_val, :]


    model = model.fit(feature_train, label_train)
    label_est_train = model.predict(feature_train)
    label_est_val = model.predict(feature_val)

    rmse_train = np.sqrt(mean_squared_error(label_train, label_est_train))
    rmse_val = np.sqrt(mean_squared_error(label_val, label_est_val))
#     print('\t', rmse_train, rmse_val)
    
    
    mae_train, _ = get_MAE(label_train, label_est_train)
    mape_train, _ = get_MAPE(label_train, label_est_train)

    mae_val, _ = get_MAE(label_val, label_est_val)
    mape_val, _ = get_MAPE(label_val, label_est_val)
    # rmse_val = np.sqrt(mean_squared_error(SpO2_val, SpO2_est_val))
    if subject_id==112:
        sys.exit()

    Rsquared_train = get_CoeffDeterm(label=label_train, predictions=label_est_train)
    Rsquared_val = get_CoeffDeterm(label=label_val, predictions=label_est_val)
#     print('\t', Rsquared_train, Rsquared_val)
    

    df_performance.append(pd.DataFrame({
       'label_est_val': label_est_val, 
       'label_val': label_val, 
       'subject_id': meta_val[:,0], 
       'task_id': meta_val[:,1], 
        'Rsquared_train': Rsquared_train,
        'Rsquared_val': Rsquared_val,
        'rmse_train': rmse_train,
        'rmse_val': rmse_val,
        'mae_train': mae_train,
        'mae_val': mae_val,
        'mape_train': mape_train,
        'mape_val': mape_val,
    }))
#     est_concat.append(label_est_val)
#     label_concat.append(label_val)

    # PCC_train, _ = pearsonr(SpO2_train, SpO2_est_train)
    # PCC_val, _ = pearsonr(SpO2_val, SpO2_est_val)

In [ ]:
df_performance = pd.concat(df_performance)
df_performance

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,2), dpi=80)

ax.plot(df_performance['label_est_val'].values, color='r', alpha=0.5)
ax.plot(df_performance['label_val'].values, color='b', alpha=0.5)

In [ ]:
Rsquared_val_mean = df_performance[['Rsquared_val', 'subject_id']].drop_duplicates()['Rsquared_val'].mean()
Rsquared_val_std = df_performance[['Rsquared_val', 'subject_id']].drop_duplicates()['Rsquared_val'].std()

rmse_val_mean = df_performance[['rmse_val', 'subject_id']].drop_duplicates()['rmse_val'].mean()
rmse_val_std = df_performance[['rmse_val', 'subject_id']].drop_duplicates()['rmse_val'].std()

mae_val_mean = df_performance[['mae_val', 'subject_id']].drop_duplicates()['mae_val'].mean()
mae_val_std = df_performance[['mae_val', 'subject_id']].drop_duplicates()['mae_val'].std()

mape_val_mean = df_performance[['mape_val', 'subject_id']].drop_duplicates()['mape_val'].mean()
mape_val_std = df_performance[['mape_val', 'subject_id']].drop_duplicates()['mape_val'].std()

In [ ]:
N_sub = len(df_performance['subject_id'].unique())

N_window = df_performance.shape[0]

In [ ]:
N_sub, N_window, df_performance['subject_id'].unique(), df_performance['task_id'].unique()

In [ ]:
df_performance[['rmse_val', 'subject_id']].drop_duplicates()[['subject_id', 'rmse_val']]

In [ ]:
df_performance[['mape_val', 'subject_id']].drop_duplicates()[['subject_id', 'mape_val']]

In [ ]:
df_performance['task_id'].unique()

In [ ]:
unit = unit_dict[label_name[0].split('_')[0]]

fig, ax = plt.subplots(1,1, figsize=(8,8,), dpi=80)
sns.scatterplot(data=df_performance, x='label_val', y='label_est_val', hue='subject_id', ax=ax, palette=subject_palette)
# sns.scatterplot(data=df_performance, x='label_val', y='label_est_val', hue='task_id', ax=ax, palette=task_palette)

# ax.scatter(df_performance['label_val'].values, df_performance['label_est_val'].values, alpha=0.5)
textstr = 'RMSE={:.3f}±{:.3f} {}\nMAE={:.3f}±{:.3f} {}\nMAPE={:.3f}±{:.3f} %\nCoeffDetermination={:.2f}±{:.2f}\nN_sub = {}\nN_window = {}' \
.format(rmse_val_mean, rmse_val_std, unit, mae_val_mean, mae_val_std, unit, mape_val_mean*100, mape_val_std*100, Rsquared_val_mean, Rsquared_val_std, N_sub, N_window )

props = dict(boxstyle='round', facecolor='white', alpha=0.5)
fontsize = 20
# place a text box in upper left in axes coords
ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=fontsize*0.7,
        verticalalignment='top', bbox=props)

ax_min = min(df_performance['label_val'].values.min(), df_performance['label_est_val'].values.min())
ax_max = min(df_performance['label_val'].values.max(), df_performance['label_est_val'].values.max())
ax.plot([ax_min, ax_max], [ax_min, ax_max])
ax.set_xlim([ax_min, ax_max])
ax.set_ylim([ax_min, ax_max])
ax.legend(bbox_to_anchor=(1.15, 1.05), frameon=True, fontsize=fontsize)
ax.set_xlabel('{} Actual\n{}'.format(label_name[0], unit), fontsize=fontsize)
ax.set_ylabel('{} Estiamted\n{}'.format(label_name[0], unit), fontsize=fontsize)

# convert meta to int array

In [ ]:
plt.plot(data_all[0,0,:])

In [ ]:
plt.plot(data_all[0,1,:])

In [ ]:
plt.plot(data_all[100,2,:])

In [ ]:
list_signal

In [ ]:
data_all.shape, meta_all.shape, label_all.shape, feature_all.shape

# save all data

In [ ]:
stage3_dict = {
    'list_feature': list_feature,
    'list_output': list_output,
    'list_meta': list_meta,
    'list_signal': list_signal,
    'FS_RESAMPLE_DL': FS_RESAMPLE_DL,
    'subject_ids': np.unique(meta_all[:,0]),
    'task_ids': np.unique(meta_all[:,1]),
    'window_size': window_size,
    'overlap': overlap,
}

In [ ]:
data_saver(stage3_dict, 'stage3_dict', outputdir)


In [ ]:
data_saver(data_all, 'data', outputdir)
data_saver(feature_all, 'feature', outputdir)
data_saver(label_all, 'label', outputdir)
data_saver(meta_all, 'meta', outputdir)

In [ ]:
sys.exit()

In [ ]:
j = 5

plt.figure(figsize=(30,5))
plt.plot(data_all[j,0,:])
plt.show()
# plt.plot(ecg_filt)

In [ ]:
list_output

In [ ]:
# indices = np.where(labels==1)[0]
# np.random.shuffle(indices)
# indices = indices[0:examples_N]

# # get their FT
# data_FT = []
# for i_plot in range(examples_N):
#     v_acc = data[indices[i_plot],:,:]
#     yf = np.zeros((v_acc.shape[0],v_acc.shape[1]-1))
#     for i_axis in range(v_acc.shape[1]-1): # don't do it for HR
#         yf[:,i_axis] = scipy.fftpack.fft(data[indices[i_plot],:,i_axis])
#     T = 1/sampling_freq
#     N = v_acc.shape[0]
#     xf = np.linspace(0.0, 1.0/(2.0*T), N/2)
#     data_FT.append({'freq':xf, 'magnitude': yf})
    
# # plot
# plot_indices(data, labels, i_seizures, indices, visual_resultsdir, normalize_window_wise=True)

In [ ]:
label_all[j, 0]

In [ ]:



v_arr = data_all[j,1,:]
# yf = np.zeros(v_arr.shape[0])
# for i_axis in range(v_arr.shape[1]-1): # don't do it for HR
T = 1/FS_RESAMPLE_DL
N = v_arr.shape[0]
yf = scipy.fftpack.fft(v_arr)
yf = 2.0/N * np.abs(yf[:N//2])

xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
# data_FT.append({'freq':xf, 'magnitude': yf})

In [ ]:
plt.plot(xf, yf)

In [ ]:
plt.figure(figsize=(30,5))
plt.plot(data_all[j,1,:])
plt.show()
# plt.plot(ecg_filt)

In [ ]:
# plt.figure(figsize=(100,10))
# plt.plot(t_arr, ecg_filt)
# plt.plot(time_interp, ecg_filt_resample)

In [ ]:
# data_sub.shape, label_sub.shape

In [ ]:
outputdir

In [ ]:
sys.exit()

# Split data into train, val

# subject specific model, leave-one-task-out

In [ ]:
data_all.shape, meta_all.shape, label_all.shape

In [ ]:
def split_train_val_4sub(data_sub, meta_sub, label_sub):

    for task_id in np.unique(meta_sub[:,1]):

        mask_train = meta_all[:,1]!=task_id
        mask_val = meta_all[:,1]==task_id


        data_train = data_all[mask_train,:,:]
        data_val = data_all[mask_val,:,:]

        label_train = label_all[mask_train,:]
        label_val = label_all[mask_val,:]

        meta_train = meta_all[mask_train,:]
        meta_val = meta_all[mask_val,:]

        outputdir_CV = outputdir_sub + 'CV{}/'.format(task_id)

        outputdir_train = outputdir_CV+'train/'
        if not os.path.exists(outputdir_train):
            os.makedirs(outputdir_train)
        data_saver(data_train, 'data', outputdir_train)
        data_saver(label_train, 'label', outputdir_train)
        data_saver(meta_train, 'meta', outputdir_train)

        outputdir_val = outputdir_CV+'val/'
        if not os.path.exists(outputdir_val):
            os.makedirs(outputdir_val)

        data_saver(data_val, 'data', outputdir_val)
        data_saver(label_val, 'label', outputdir_val)
        data_saver(meta_val, 'meta', outputdir_val)

    print('finish saving!')

In [ ]:
for subject_id in np.unique(meta_all[:,0]):
    outputdir_sub = outputdir+str(subject_id)+'/'
    
    mask_all = meta_all[:,0]==subject_id
    
    data_sub = data_all[mask_all, :, :]
    meta_sub = meta_all[mask_all, :]
    label_sub = label_all[mask_all, :]
    split_train_val_4sub(data_sub, meta_sub, label_sub)
    

In [ ]:
sys.exit()

In [ ]:
# outputdir_CV

In [ ]:
print(np.unique(meta_all[:,0]))
print(np.unique(meta_all[:,-1]))

In [ ]:
# LO_name = 'Recovery 6MWT'
# mask_train = meta_sub[:,1]!=LO_name
# mask_val = meta_sub[:,1]==LO_name


LO_name = 110
mask_train = meta_sub[:,0]!=LO_name
mask_val = meta_sub[:,0]==LO_name


data_train = data_sub[mask_train,:,:]
data_val = data_sub[mask_val,:,:]

label_train = label_sub[mask_train,:]
label_val = label_sub[mask_val,:]

meta_train = meta_sub[mask_train,:]
meta_val = meta_sub[mask_val,:]



In [ ]:
print(mask_train.sum(), mask_val.sum())

In [ ]:
outputdir_train = outputdir+'train/'
if not os.path.exists(outputdir_train):
    os.makedirs(outputdir_train)

data_saver(data_train, 'data', outputdir_train)
data_saver(label_train, 'label', outputdir_train)
data_saver(meta_train, 'meta', outputdir_train)

In [ ]:
outputdir_val = outputdir+'val/'
if not os.path.exists(outputdir_val):
    os.makedirs(outputdir_val)

data_saver(data_val, 'data', outputdir_val)
data_saver(label_val, 'label', outputdir_val)
data_saver(meta_val, 'meta', outputdir_val)


In [ ]:
# outputdir_train